In [7]:
import pandas_datareader.data as web
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps, resample_apply
import talib as ta
import importlib
import os
import platform
import psutil
import pandas as pd

# start = datetime.date(2021,1,1)
start = datetime.datetime.today() + relativedelta.relativedelta(years=-1)
end = datetime.date.today()
#https://www.sbisec.co.jp/ETGate/WPLETmgR001Control?OutSide=on&getFlg=on&burl=search_domestic&cat1=domestic&cat2=none&dir=info&file=domestic_info150818_01.html
# data = web.DataReader('7203.T', 'yahoo', start, end)
# data = web.DataReader('SPYD', 'yahoo', start, end)
# data = web.DataReader('1321.T', 'yahoo', start, end)
# data = web.DataReader('1570.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均レバレッジ・インデックス連動型上場投信
# data = web.DataReader('1357.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均ダブルインバース・インデックス連動型上場投信
# data = web.DataReader('BTC-JPY', 'yahoo', start, end)
# data = web.DataReader('^N225', 'yahoo', start, end)
# data = web.DataReader('1655.T', 'yahoo', start, end)   #iシェアーズ S&P500 米国株 ETF　→　取得されるデータがおかしい？？
# data = web.DataReader('2558.T', 'yahoo', start, end)    #MAXIS 米国株式(S&P500)上場投信
# data = web.DataReader('2631.T', 'yahoo', start, end)    #MAXIS ナスダック100上場投信
data = web.DataReader('2559.T', 'yahoo', start, end)      #MAXIS 全世界株式(オール・カントリー)上場投信
data = data.astype("double")
print(data)

initial = 10_000_000
fname = f"html/plot-"

#position
#0 ... display "only long" result
#1 ... display both "only long" and "long and short" result
position = 1

df_returns = pd.DataFrame()

               High      Low     Open    Close   Volume  Adj Close
Date                                                              
2021-09-21  13150.0  12970.0  13050.0  13020.0  39835.0    13020.0
2021-09-22  13010.0  12840.0  12960.0  13000.0  13138.0    13000.0
2021-09-24  13360.0  13280.0  13310.0  13300.0   6117.0    13300.0
2021-09-27  13400.0  13330.0  13330.0  13370.0   6822.0    13370.0
2021-09-28  13400.0  13290.0  13330.0  13360.0   5822.0    13360.0
...             ...      ...      ...      ...      ...        ...
2022-09-12  14725.0  14635.0  14670.0  14710.0   8392.0    14710.0
2022-09-13  14845.0  14800.0  14845.0  14820.0   3573.0    14820.0
2022-09-14  14515.0  14375.0  14430.0  14390.0  12573.0    14390.0
2022-09-15  14410.0  14325.0  14405.0  14400.0   4748.0    14400.0
2022-09-16  14215.0  14145.0  14190.0  14200.0   8385.0    14200.0

[244 rows x 6 columns]


**ボリンジャーバンドとRSI組み合わせ**

In [8]:
import BacktestingStrategies.Strategy_BBDandRSI as btstBBandRsi
importlib.reload(btstBBandRsi)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstBBandRsi.BBDandRSI, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html")
df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)
output=bt.optimize(\
    upper_bound = range(50, 85, 5),
    lower_bound = range(10, 45, 5),
    rsi_window=range(10, 30, 2),\
    maximize='Equity Final [$]',\
    method='grid',\
    max_tries = 1000)
print(output)
display(output._trades)
display(output._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 
df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
df_temp["_strategy"] = str(output._strategy) + "_optimize"
df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstBBandRsi.BBDandRSI_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )
    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html")
    df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
    df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

    # #最適化
    p = psutil.Process()
    p.nice(psutil.HIGH_PRIORITY_CLASS)
    output=bt.optimize(\
        upper_bound = range(50, 85, 5),
        lower_bound = range(10, 45, 5),
        rsi_window=range(10, 30, 2),\
        maximize='Equity Final [$]',\
        method='grid',\
        max_tries = 1000)
    print(output)
    display(output._trades)
    display(output._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")
    df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
    df_temp["_strategy"] = str(output._strategy) + "_optimize"
    df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

Start                     2021-09-21 00:00:00
End                       2022-09-16 00:00:00
Duration                    360 days 00:00:00
Exposure Time [%]                        50.0
Equity Final [$]                10597000.5835
Equity Peak [$]                 10871387.6725
Return [%]                           5.970006
Buy & Hold Return [%]                 9.06298
Return (Ann.) [%]                    6.171665
Volatility (Ann.) [%]               16.317335
Sharpe Ratio                         0.378228
Sortino Ratio                        0.563591
Calmar Ratio                         0.623425
Max. Drawdown [%]                   -9.899608
Avg. Drawdown [%]                    -4.15504
Max. Drawdown Duration      177 days 00:00:00
Avg. Drawdown Duration       39 days 00:00:00
# Trades                                    3
Win Rate [%]                        66.666667
Best Trade [%]                       4.128757
Worst Trade [%]                     -1.340035
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,715,47,71,13978.8545,14420.0,315419.0325,0.031558,2021-11-30,2022-01-05,36 days
1,768,83,123,13416.0825,13970.0,425408.6400,0.041288,2022-01-24,2022-03-24,59 days
2,758,148,203,14159.7455,13970.0,-143827.0890,-0.013400,2022-04-28,2022-07-21,84 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 490 configurations.
  output = _optimize_grid()


Start                     2021-09-21 00:00:00
End                       2022-09-16 00:00:00
Duration                    360 days 00:00:00
Exposure Time [%]                   52.868852
Equity Final [$]               11809690.25725
Equity Peak [$]                11809690.25725
Return [%]                          18.096903
Buy & Hold Return [%]                 9.06298
Return (Ann.) [%]                   18.742717
Volatility (Ann.) [%]               17.919542
Sharpe Ratio                         1.045937
Sortino Ratio                        1.805541
Calmar Ratio                         1.891408
Max. Drawdown [%]                   -9.909401
Avg. Drawdown [%]                   -3.511428
Max. Drawdown Duration       41 days 00:00:00
Avg. Drawdown Duration       17 days 00:00:00
# Trades                                    5
Win Rate [%]                            100.0
Best Trade [%]                       4.914635
Worst Trade [%]                      1.696743
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,715,47,71,13978.85450,14420.0,315419.03250,0.031558,2021-11-30,2022-01-05,36 days
1,753,81,122,13697.46850,14055.0,269221.21950,0.026102,2022-01-20,2022-03-23,62 days
2,747,148,174,14159.74550,14400.0,179470.11150,0.016967,2022-04-28,2022-06-09,42 days
3,808,177,203,13315.58750,13970.0,528765.30000,0.049146,2022-06-14,2022-07-21,37 days
4,795,233,240,14194.91875,14845.0,516814.59375,0.045797,2022-09-02,2022-09-13,11 days


...

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-714,29,71,13990.40300,14420.0,-306732.25800,-0.030707,2021-11-02,2022-01-05,64 days
1,722,83,123,13416.08250,13970.0,399928.43500,0.041288,2022-01-24,2022-03-24,59 days
2,-711,125,126,14179.46250,14430.0,-178132.16250,-0.017669,2022-03-28,2022-03-29,1 days
3,-681,127,128,14552.60625,14450.0,69874.85625,0.007051,2022-03-30,2022-03-31,1 days
4,705,148,203,14159.74550,13970.0,-133770.57750,-0.013400,2022-04-28,2022-07-21,84 days
5,-708,204,216,13905.82375,14220.0,-222436.78500,-0.022593,2022-07-22,2022-08-09,18 days
6,-669,220,221,14373.49725,14500.0,-84630.33975,-0.008801,2022-08-16,2022-08-17,1 days
7,-659,222,224,14473.00225,14500.0,-17791.51725,-0.001865,2022-08-18,2022-08-22,4 days
8,-663,241,243,14358.57150,14190.0,111762.90450,0.011740,2022-09-14,2022-09-16,2 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 490 configurations.
  output = _optimize_grid()


Start                     2021-09-21 00:00:00
End                       2022-09-16 00:00:00
Duration                    360 days 00:00:00
Exposure Time [%]                   59.836066
Equity Final [$]                11489237.8215
Equity Peak [$]                11890901.86125
Return [%]                          14.892378
Buy & Hold Return [%]                 9.06298
Return (Ann.) [%]                   15.416521
Volatility (Ann.) [%]               16.949892
Sharpe Ratio                         0.909535
Sortino Ratio                        1.675974
Calmar Ratio                         2.098777
Max. Drawdown [%]                   -7.345478
Avg. Drawdown [%]                   -3.159682
Max. Drawdown Duration      115 days 00:00:00
Avg. Drawdown Duration       22 days 00:00:00
# Trades                                   14
Win Rate [%]                        64.285714
Best Trade [%]                       5.037748
Worst Trade [%]                     -2.511728
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,715,47,54,13978.85450,14075.0,68744.03250,0.006878,2021-11-30,2021-12-09,9 days
1,737,62,67,13652.24575,14270.0,455284.88225,0.045249,2021-12-21,2021-12-28,7 days
2,-733,71,76,14348.62100,14195.0,112604.19300,0.010706,2022-01-05,2022-01-13,8 days
3,781,82,96,13617.07250,14000.0,299066.37750,0.028121,2022-01-21,2022-02-10,20 days
4,834,104,121,13109.57275,13770.0,550796.32650,0.050377,2022-02-24,2022-03-22,26 days
5,-826,123,125,13900.84850,14250.0,-288399.13900,-0.025117,2022-03-24,2022-03-28,4 days
6,-779,126,127,14358.57150,14625.0,-207547.80150,-0.018555,2022-03-29,2022-03-30,1 days
7,-764,128,168,14378.47250,13940.0,334992.99000,0.030495,2022-03-31,2022-06-01,62 days
8,841,178,203,13466.33000,13970.0,423586.47000,0.037402,2022-06-15,2022-07-21,36 days
9,-844,204,207,13905.82375,13875.0,26015.24500,0.002217,2022-07-22,2022-07-27,5 days


<Strategy BBDandRSI_WithShortPosition(upper_bound=55,lower_bound=40,rsi_window=12)>

**MACDとRSIの組み合わせ**

In [9]:
import BacktestingStrategies.Strategy_MACDandRSI as btstMACDandRsi
importlib.reload(btstMACDandRsi)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstMACDandRsi.MACDandRSI, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html")
df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)
output=bt.optimize(\
    MACDshort=range(10, 25, 5),\
    MACDlong=range(10, 75, 10),\
    MACDsignal=range(10, 15, 5),\
    upper_bound = range(50, 85, 5),\
    lower_bound = range(10, 45, 5),\
    rsi_window=range(10, 30, 2),\
    maximize='Equity Final [$]',\
    method='grid',\
    constraint = lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
    max_tries = 1000)
print(output)
display(output._trades)
display(output._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html")
df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
df_temp["_strategy"] = str(output._strategy) + "_optimize"
df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstMACDandRsi.MACDandRSI_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html")
    df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
    df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

    # #最適化
    p = psutil.Process()
    p.nice(psutil.HIGH_PRIORITY_CLASS)
    output=bt.optimize(\
        MACDshort=range(10, 25, 5),\
        MACDlong=range(10, 75, 10),\
        MACDsignal=range(10, 15, 5),\
        upper_bound = range(50, 85, 5),
        lower_bound = range(10, 45, 5),
        rsi_window=range(10, 30, 2),\
        maximize='Equity Final [$]',\
        method='grid',\
        constraint = lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
        max_tries = 1000)
    print(output)
    display(output._trades)
    display(output._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")
    df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
    df_temp["_strategy"] = str(output._strategy) + "_optimize"
    df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

Start                     2021-09-21 00:00:00
End                       2022-09-16 00:00:00
Duration                    360 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                   10000000.0
Equity Peak [$]                    10000000.0
Return [%]                                0.0
Buy & Hold Return [%]                 9.06298
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 550 configurations.
  output = _optimize_grid()


Start                     2021-09-21 00:00:00
End                       2022-09-16 00:00:00
Duration                    360 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                   10000000.0
Equity Peak [$]                    10000000.0
Return [%]                                0.0
Buy & Hold Return [%]                 9.06298
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration


...

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-717,78,103,13935.67525,13200.0,527479.15425,0.052791,2022-01-17,2022-02-22,36 days
1,-741,137,145,14194.38825,14200.0,-4158.30675,-0.000395,2022-04-13,2022-04-25,12 days
2,-789,178,222,13333.67000,14545.0,-955739.37000,-0.090847,2022-06-15,2022-08-18,64 days
3,-663,224,227,14428.22500,14350.0,51863.17500,0.005422,2022-08-22,2022-08-25,3 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 552 configurations.
  output = _optimize_grid()


Start                     2021-09-21 00:00:00
End                       2022-09-16 00:00:00
Duration                    360 days 00:00:00
Exposure Time [%]                   26.639344
Equity Final [$]               10800841.92425
Equity Peak [$]                 11050544.1735
Return [%]                           8.008419
Buy & Hold Return [%]                 9.06298
Return (Ann.) [%]                    8.281579
Volatility (Ann.) [%]               10.148024
Sharpe Ratio                         0.816078
Sortino Ratio                        1.452224
Calmar Ratio                         1.256221
Max. Drawdown [%]                   -6.592456
Avg. Drawdown [%]                   -3.207717
Max. Drawdown Duration       88 days 00:00:00
Avg. Drawdown Duration       36 days 00:00:00
# Trades                                    6
Win Rate [%]                        83.333333
Best Trade [%]                       6.391332
Worst Trade [%]                     -0.186539
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-717,78,104,13935.67525,13045.0,638614.15425,0.063913,2022-01-17,2022-02-24,38 days
1,-731,127,128,14552.60625,14450.0,75005.16875,0.007051,2022-03-30,2022-03-31,1 days
2,-745,141,146,14373.49725,14275.0,73380.45125,0.006853,2022-04-19,2022-04-26,7 days
3,-803,179,191,13428.19975,13390.0,30674.39925,0.002845,2022-06-16,2022-07-04,18 days
4,-747,222,224,14473.00225,14500.0,-20167.31925,-0.001865,2022-08-18,2022-08-22,4 days
5,-760,230,243,14194.38825,14190.0,3335.07000,0.000309,2022-08-30,2022-09-16,17 days


<Strategy MACDandRSI_WithShortPosition(MACDshort=15,MACDlong=30,MACDsignal=10,upper_bound=75,lower_bound=40,rsi_window=10)>

**MACDとボリンジャーバンドの組み合わせ**

In [10]:
import BacktestingStrategies.Strategy_MACDandBBD as btstMACDandBB
importlib.reload(btstMACDandBB)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstMACDandBB.MACDandBBD, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html")
df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)
output=bt.optimize(\
    MACDshort=range(10, 25, 5),\
    MACDlong=range(10, 75, 10),\
    MACDsignal=range(10, 15, 5),\
    maximize='Equity Final [$]',\
    method='grid',\
    constraint = lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
    max_tries = 1000)
print(output)
display(output._trades)
display(output._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html")
df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
df_temp["_strategy"] = str(output._strategy) + "_optimize"
df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstMACDandBB.MACDandBBD_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )
    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html")
    df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
    df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

    # #最適化
    p = psutil.Process()
    p.nice(psutil.HIGH_PRIORITY_CLASS)
    output=bt.optimize(\
        MACDshort=range(10, 25, 5),\
        MACDlong=range(10, 75, 10),\
        MACDsignal=range(10, 15, 5),\
        maximize='Equity Final [$]',\
        method='grid',\
        constraint = lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
        max_tries = 1000)
    print(output)
    display(output._trades)
    display(output._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")
    df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
    df_temp["_strategy"] = str(output._strategy) + "_optimize"
    df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

Start                     2021-09-21 00:00:00
End                       2022-09-16 00:00:00
Duration                    360 days 00:00:00
Exposure Time [%]                   20.901639
Equity Final [$]                 9821556.8375
Equity Peak [$]                 10211815.3595
Return [%]                          -1.784432
Buy & Hold Return [%]                 9.06298
Return (Ann.) [%]                   -1.842395
Volatility (Ann.) [%]                6.117486
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -4.382546
Avg. Drawdown [%]                   -2.769047
Max. Drawdown Duration      223 days 00:00:00
Avg. Drawdown Duration       85 days 00:00:00
# Trades                                    4
Win Rate [%]                             50.0
Best Trade [%]                       2.214154
Worst Trade [%]                     -3.356333
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,690,71,78,14491.3790,14005.0,-335601.5100,-0.033563,2022-01-05,2022-01-17,12 days
1,688,123,137,14039.1515,14265.0,155383.7680,0.016087,2022-03-24,2022-04-13,20 days
2,699,203,227,14039.1515,14350.0,217283.1015,0.022142,2022-07-21,2022-08-25,35 days
3,692,241,243,14501.4285,14190.0,-215508.5220,-0.021476,2022-09-14,2022-09-16,2 days


Start                     2021-09-21 00:00:00
End                       2022-09-16 00:00:00
Duration                    360 days 00:00:00
Exposure Time [%]                   21.311475
Equity Final [$]                  9931216.838
Equity Peak [$]                 10211815.3595
Return [%]                          -0.687832
Buy & Hold Return [%]                 9.06298
Return (Ann.) [%]                   -0.710303
Volatility (Ann.) [%]                6.282947
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -4.382546
Avg. Drawdown [%]                   -2.411095
Max. Drawdown Duration      223 days 00:00:00
Avg. Drawdown Duration       85 days 00:00:00
# Trades                                    4
Win Rate [%]                             50.0
Best Trade [%]                       3.353824
Worst Trade [%]                     -3.356333
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,690,71,78,14491.3790,14005.0,-335601.5100,-0.033563,2022-01-05,2022-01-17,12 days
1,688,123,137,14039.1515,14265.0,155383.7680,0.016087,2022-03-24,2022-04-13,20 days
2,699,203,228,14039.1515,14510.0,329123.1015,0.033538,2022-07-21,2022-08-26,36 days
3,699,241,243,14501.4285,14190.0,-217688.5215,-0.021476,2022-09-14,2022-09-16,2 days


<Strategy MACDandBBD(MACDshort=15,MACDlong=20,MACDsignal=10)>

Start                     2021-09-21 00:00:00
End                       2022-09-16 00:00:00
Duration                    360 days 00:00:00
Exposure Time [%]                   43.032787
Equity Final [$]                 9291463.9595
Equity Peak [$]                   10064723.59
Return [%]                           -7.08536
Buy & Hold Return [%]                 9.06298
Return (Ann.) [%]                   -7.308966
Volatility (Ann.) [%]               12.465721
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -13.948222
Avg. Drawdown [%]                   -9.755868
Max. Drawdown Duration      192 days 00:00:00
Avg. Drawdown Duration      128 days 00:00:00
# Trades                                    5
Win Rate [%]                             40.0
Best Trade [%]                       5.164366
Worst Trade [%]                      -8.60578
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,690,71,78,14491.3790,14005.0,-335601.5100,-0.033563,2022-01-05,2022-01-17,12 days
1,-735,103,137,13134.6600,14265.0,-830799.9000,-0.086058,2022-02-22,2022-04-13,50 days
2,-625,145,178,14129.7100,13400.0,456068.7500,0.051644,2022-04-25,2022-06-15,51 days
3,661,203,227,14039.1515,14350.0,205470.8585,0.022142,2022-07-21,2022-08-25,35 days
4,654,241,243,14501.4285,14190.0,-203674.2390,-0.021476,2022-09-14,2022-09-16,2 days


Start                     2021-09-21 00:00:00
End                       2022-09-16 00:00:00
Duration                    360 days 00:00:00
Exposure Time [%]                   57.786885
Equity Final [$]                 9552778.7045
Equity Peak [$]                  10369795.366
Return [%]                          -4.472213
Buy & Hold Return [%]                 9.06298
Return (Ann.) [%]                   -4.615407
Volatility (Ann.) [%]               14.076523
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -14.569575
Avg. Drawdown [%]                   -5.193745
Max. Drawdown Duration      270 days 00:00:00
Avg. Drawdown Duration       77 days 00:00:00
# Trades                                    5
Win Rate [%]                             60.0
Best Trade [%]                       4.492024
Worst Trade [%]                     -9.896229
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-711,36,78,14050.10600,14005.0,32070.3660,0.003210,2021-11-12,2022-01-17,66 days
1,-772,104,137,12980.42725,14265.0,-991690.1630,-0.098962,2022-02-24,2022-04-13,48 days
2,-639,145,179,14129.71000,13495.0,405579.6900,0.044920,2022-04-25,2022-06-16,52 days
3,672,203,228,14039.15150,14510.0,316410.1920,0.033538,2022-07-21,2022-08-26,36 days
4,673,241,243,14501.42850,14190.0,-209591.3805,-0.021476,2022-09-14,2022-09-16,2 days


<Strategy MACDandBBD_WithShortPosition(MACDshort=15,MACDlong=20,MACDsignal=10)>

In [11]:
df_returns_sorted = df_returns[["_strategy", "Start", "End", "Duration", "Exposure Time [%]", "Equity Final [$]", "Equity Peak [$]", "Return [%]", "Buy & Hold Return [%]", "Return (Ann.) [%]", "Volatility (Ann.) [%]", "Sharpe Ratio", "Sortino Ratio", "Calmar Ratio", "Max. Drawdown [%]", "Avg. Drawdown [%]", "Max. Drawdown Duration", "Avg. Drawdown Duration", "# Trades", "Win Rate [%]", "Best Trade [%]", "Worst Trade [%]", "Avg. Trade [%]", "Max. Trade Duration", "Avg. Trade Duration", "Profit Factor", "Expectancy [%]", "SQN", 
]].sort_values(by = "Equity Final [$]", ascending = False)
df_returns_sorted.to_csv("CombinationStrategies.csv")